<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Applying Fourier Transform on signal series.</b>
    <br>
<b style = 'font-size:24px;font-family:Arial;color:#E37C4D'>Discrete Fast fourier transform and Convolve functions </b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <b>Fourier transform</b> is one of the most important algorithms in all of 
science, mathematics, and communications technology.
<br>
<p style = 'font-size:16px;font-family:Arial'> The Fourier Theorem states that any signal can be perfectly represented as a sum of sine waves, each sine wave having its own phase, frequency, and amplitude. This provides an efficient way to <b>transform a signal (e.g. a radio wave) from the time domain to the frequency domain</b>. The time domain is what we are used to seeing (e.g.a radio wave is a fluctuation of signal amplitude over time). The frequency domain shows how much energy is in the signal at each frequency (sine wave speed). This allows us to view the individual components of a signal, and therefore provides a better understanding of complicated signals.

<br>

<p style = 'font-size:16px;font-family:Arial'>There are four parts to a wave:
wavelength, period, frequency, and amplitude</p>
<img id="signal wave" src="images/signal.png" alt="signal wave" width="400" />
<p style = 'font-size:16px;font-family:Arial'>The Angular Frequency is ω = 2π × f</p>

### Applications for FFT are
<p style = 'font-size:16px;font-family:Arial'>
<ul>
            <li  style = 'font-size:16px;font-family:Arial'>Digital Signal Processing</li>
            <li style = 'font-size:16px;font-family:Arial'>Noise reduction in signals</li>
            <li  style = 'font-size:16px;font-family:Arial'>Seasonality detection in timeseries</li>
            <li  style = 'font-size:16px;font-family:Arial'>Image Processing</li>
        </ul>    
</p>

<p style = 'font-size:16px;font-family:Arial'>This demo works with sample data generated in the notebook and creates a table under Demo_User. You may need to use the Data Dictionary "Manage Your Space" routine to cleanup tables you no longer need. 
    
<p style = 'font-size:16px;font-family:Arial'>Use the link below to access the data preparation notebook.

[Click Here to Manage Your Space](../Data_Dictionary/Data_Dictionary.ipynb#Manage_Your_Space)

<hr>
<p style = 'font-size:16px;font-family:Arial;'>Let's do a simple example on sample data on how the signals with random noise looks like and how do we recognize noise using FFT.</p>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>This steps connects with Vantage. You might be prompted to enter the password.</p>

In [5]:
%connect local

Password: ···


Success: 'local' connection established and activated for user 'demo_user'


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Create the dataset</b>
<p style = 'font-size:16px;font-family:Arial'>We will create two signals with 400 datapoints. One pure signal which will have 4sinusoidal frequencies of 2, 25, 50 & 100 Hz and other which will have random noise added as well. 

In [5]:
CREATE TABLE genData(ID integer, row_i integer, MAGNITUDE float);

Success: 0 rows affected

In [6]:
INSERT INTO genData values(1, 1, 1.0);
INSERT INTO genData SELECT ID, row_i+1, MAGNITUDE+1.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+2, MAGNITUDE+2.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+4, MAGNITUDE+4.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+8, MAGNITUDE+8.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+16, MAGNITUDE+16.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+32, MAGNITUDE+32.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+64, MAGNITUDE+64.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+128, MAGNITUDE+128.0 FROM genData;
INSERT INTO genData SELECT ID, row_i+256, MAGNITUDE+256.0 FROM genData;

Success: 1 rows affected

Success: 1 rows affected

Success: 2 rows affected

Success: 4 rows affected

Success: 8 rows affected

Success: 16 rows affected

Success: 32 rows affected

Success: 64 rows affected

Success: 128 rows affected

Success: 256 rows affected

In [7]:
CREATE TABLE GenDataSinusoids AS (SELECT * FROM GenData) WITH DATA;

Success: 0 rows affected

In [8]:
--To get only 400 data points
DELETE FROM GenDataSinusoids WHERE ROW_I>400.0;

Success: 112 rows affected

In [9]:
/*************************************************************************/
/*  Generate 400 Samplepoint Series which is the sum of 4 Sinusoids      */
/*  Sinusoids have frequencies (k) of: 2, 25, 50, 100                    */
/*************************************************************************/
EXECUTE FUNCTION INTO VOLATILE ART(GeneratedData)
TD_GENSERIES4FORMULA(
  SERIES_SPEC(TABLE_NAME(GenDataSinusoids), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
  PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))
  ),
  FUNC_PARAMS(Formula('Y = 4.0*sin(6.283185307179586476*X1/200.0) + 4.0*sin(6.283185307179586476*X1/16.0 + 3.14159265358979323/4.0) + 4.0*sin(6.283185307179586476*X1/8.0 + 3.14159265358979323/2.0) + 4.0*sin(6.283185307179586476*X1/4.0 + 3.14159265358979323/8.0)'))
);

Success: 400 rows affected

In [10]:
CREATE TABLE SINUSOIDS4 AS (SELECT * FROM GeneratedData) WITH DATA;

Success: 0 rows affected

In [11]:
select * from SINUSOIDS4 sample 10;

,ID,ROW_I,MAGNITUDE
1,1,6,-1.525251769173146
2,1,158,-3.1775320185463927
3,1,182,-4.433990397760874
4,1,206,1.5362156897473827
5,1,210,-1.4734754437650306
6,1,243,4.28830985712123
7,1,258,1.012747617961586
8,1,276,5.0430721960539575
9,1,292,3.2703977003455544
10,1,354,-6.779180487126649


<p>Table SINUSOIDS4 is the table containing signal data of 4 sinusidal waves.</p><p>We can see how the signal looks like using TD_PLOT.</p>


In [12]:
/**********************************************************************************/
/*                              PLOT generated sinusoidal signal                  */
/**********************************************************************************/

EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(SINUSOIDS4), ROW_AXIS(SEQUENCE(ROW_I)), 
                SERIES_ID(ID),
                PAYLOAD(FIELDS(Magnitude), CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line'))], IMAGE('png')));

,ID,ROW_I,IMAGE
1,1,0,...


<img id="sine wave" src="images/sin4wave.png" alt="Sine wave with no noise added" width="400" />
<p style = 'font-size:16px;font-family:Arial'>Sine wave with no noise added.</p>

<p style = 'font-size:16px;font-family:Arial'>Now,let us add some <b>noise </b>in the signal

In [15]:
EXECUTE FUNCTION INTO VOLATILE ART(GeneratedData_noise)
TD_GENSERIES4FORMULA(
  SERIES_SPEC(TABLE_NAME(GenDataSinusoids), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
  PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))
  ),
  FUNC_PARAMS(Formula('Y = 4.0*sin(6.283185307179586476*X1/200.0) + 4.0*sin(6.283185307179586476*X1/16.0 + 3.14159265358979323/4.0) + 4.0*sin(6.283185307179586476*X1/8.0 + 3.14159265358979323/2.0) + 4.0*sin(6.283185307179586476*X1/4.0 + 3.14159265358979323/8.0) + (3* random(1,10)/random(1,3))'))
);

Success: 400 rows affected

In [16]:
CREATE TABLE SINUSOIDS4_noise AS (SELECT * FROM GeneratedData_noise) WITH DATA;

Success: 0 rows affected

In [17]:
/**********************************************************************************/
/*             PLOT generated sinusoidal signal with noise added                  */
/**********************************************************************************/

EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(SINUSOIDS4_noise), ROW_AXIS(SEQUENCE(ROW_I)), 
                SERIES_ID(ID),
                PAYLOAD(FIELDS(Magnitude), CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line'))], IMAGE('png')));


,ID,ROW_I,IMAGE
1,1,0,...


<img id="sine wave with noise" src="images/sine_with_noise.png" alt="Sine wave with noise added" width="400" />
<p style = 'font-size:16px;font-family:Arial'>Sine wave with noise added.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. DFFT and IDFFT</b>
<br>
<p style = font-size:16px;font-family:Arial>Discrete Fourier Transform (DFFT) & Inverse DFFT are the functions availabe in Vantage Clearscape analytics which can convert any timeseries in frequency domain and vice versa using the Vantage's UAF (unboundend array framework)</p>
<p style = 'font-size:16px;font-family:Arial'>Let us plot Fourier Transform to see if we can distinguish the main signals (sinusoids)
</p>

In [19]:
/**********************************************************************************/
/*          Applying TD_DFFT function with signal with no noise                   */
/**********************************************************************************/
EXECUTE FUNCTION INTO VOLATILE ART(DfftRaw_INTEGRAL) 
TD_DFFT(
  SERIES_SPEC( TABLE_NAME(SINUSOIDS4), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
               PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL)) ),
  FUNC_PARAMS( FREQ_STYLE("K_INTEGRAL"), HUMAN_READABLE(1) ),
  OUTPUT_FMT( CONTENT(AMPL_PHASE_RADIANS) ) );

Success: 402 rows affected

In [20]:
/* Return Fourier Coeff rows having a significant magnitude */
SELECT * FROM DfftRaw_INTEGRAL WHERE Amplitude_Magnitude > 1.0;

,ID,ROW_I,AMPLITUDE_MAGNITUDE,PHASE_MAGNITUDE
1,1,-100,800.0000000000001,-0.39269908169871437
2,1,-50,799.9999999999989,-0.7853981633974361
3,1,-25,799.9999999999977,0.3926990816987299
4,1,-2,800.0000000000001,1.539380400259
5,1,2,800.0000000000001,-1.5393804002589997
6,1,25,799.9999999999977,-0.3926990816987295
7,1,50,799.9999999999987,0.7853981633974366
8,1,100,800.0000000000001,0.39269908169871437


In [21]:
/**********************************************************************************/
/*          Applying TD_DFFT function with signal with noise                      */
/**********************************************************************************/
EXECUTE FUNCTION INTO VOLATILE ART(DfftRaw_noise) 
TD_DFFT(
  SERIES_SPEC( TABLE_NAME(SINUSOIDS4_noise), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
               PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL)) ),
  FUNC_PARAMS( FREQ_STYLE("K_INTEGRAL"), HUMAN_READABLE(1) ),
  OUTPUT_FMT( CONTENT(AMPL_PHASE_RADIANS) ) );


Success: 402 rows affected

In [22]:
/******************************************************************************************/
/*                           Plot the results of the DFFT of the Signal                  */
/******************************************************************************************/

EXECUTE FUNCTION
TD_Plot
(   SERIES_SPEC(TABLE_NAME(DfftRaw_INTEGRAL), ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(ID),
                PAYLOAD(FIELDS(Amplitude_Magnitude),CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line') )], IMAGE('png'))
);


,ID,ROW_I,IMAGE
1,1,0,...


<img id="dfft sine" src="images/dfft_sine.png" alt="DFFT of sine" width="400" />
<p style = 'font-size:16px;font-family:Arial'>DFFT of Sine wave with no noise added.</p>

In [24]:
/******************************************************************************************/
/*                           Plot the results of the DFFT of the Signal                  */
/******************************************************************************************/
EXECUTE FUNCTION
TD_Plot
(   SERIES_SPEC(TABLE_NAME(DfftRaw_noise), ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(ID),
                PAYLOAD(FIELDS(Amplitude_Magnitude),CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line') )], IMAGE('png'))
);

,ID,ROW_I,IMAGE
1,1,0,...


<img id="dfft noise" src="images/dfft_noise.png" alt="DFFT of noise" width="400" />
<p style = 'font-size:16px;font-family:Arial'>DFFT of Sine wave with noise added.</p>

<p style = 'font-size:16px;font-family:Arial'><b>We can recreate our original signals(time domain) from the dfft (frequency domain)</b></p>

In [27]:
/**********************************************************************************/
/*               TAKE INVERSE DFFT of PREVIOUS DFFT OUTPUT                        */
/**********************************************************************************/
EXECUTE FUNCTION INTO VOLATILE ART(ReturnOriginal) 
        TD_IDFFT(ART_SPEC(TABLE_NAME(DfftRaw_INTEGRAL),
                          PAYLOAD( FIELDS(AMPLITUDE_MAGNITUDE,PHASE_MAGNITUDE), 
                                   CONTENT(AMPL_PHASE_RADIANS))) );

Success: 400 rows affected

In [28]:
/******************************************************************************************/
/*                           Plot the results of the IDFFT of the Signal                  */
/******************************************************************************************/
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(ReturnOriginal), ROW_AXIS(SEQUENCE(ROW_I)), SERIES_ID(ID),
                PAYLOAD(FIELDS(Real_Amplitude_Magnitude), CONTENT(REAL))),
    FUNC_PARAMS(PLOTS[(TYPE('line'))], IMAGE('png')));

,ID,ROW_I,IMAGE
1,1,0,...


<img id="idfft sine" src="images/idfft_sine.png" alt="IDFFT of sine" width="400" />
<p style = 'font-size:16px;font-family:Arial'>original signal in time domain</p>

In [31]:
EXECUTE FUNCTION INTO VOLATILE ART(ReturnOriginal_noise) 
        TD_IDFFT(ART_SPEC(TABLE_NAME(DfftRaw_noise),
                          PAYLOAD( FIELDS(AMPLITUDE_MAGNITUDE,PHASE_MAGNITUDE), 
                                   CONTENT(AMPL_PHASE_RADIANS))) );

Success: 400 rows affected

In [32]:
/******************************************************************************************/
/*                           Plot the results of the IDFFT of the Signal                  */
/******************************************************************************************/
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(ReturnOriginal_noise), ROW_AXIS(SEQUENCE(ROW_I)), SERIES_ID(ID),
                PAYLOAD(FIELDS(Real_Amplitude_Magnitude), CONTENT(REAL))),
    FUNC_PARAMS(PLOTS[(TYPE('line'))], IMAGE('png')));

,ID,ROW_I,IMAGE
1,1,0,...


<img id="idfft noise" src="images/idfft_noise.png" alt="IDFFT of noise signal" width="400" />
<p style = 'font-size:16px;font-family:Arial'>original signal in time domain</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Signal Filtering</b>
<p style = 'font-size:16px;font-family:Arial'>We can use signal filtering to filter out the pure or significant frequences from our signals. This can be used in noise filtering also. From our random noise signal, lets filter all the frequencies except 100hz.<br><br>
In Vantage we have <b>TD_CONVOLVE</b> function which can filter one signal from another. We also have <b>TD_BINARYSERIESOP</b> function which can perform a point-wise mathematical operation (addition, subtraction, multiplication, or division) on two time series of equal size.<br> First we create a sin wave of 100hz.
</p>

In [35]:
 EXECUTE FUNCTION INTO VOLATILE ART(GeneratedData_fil)
TD_GENSERIES4FORMULA(
  SERIES_SPEC(TABLE_NAME(GenDataSinusoids), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
  PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))
  ),
  FUNC_PARAMS(Formula('Y = 4.0*sin(6.283185307179586476*X1/4.0)'))
);

Success: 400 rows affected

In [36]:
CREATE TABLE SINUSOIDS4_fil AS (SELECT * FROM GeneratedData_fil) WITH DATA;

Success: 0 rows affected

In [37]:
/**********************************************************************************/
/*                              PLOT generated sinusoidal signal                  */
/**********************************************************************************/

EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(SINUSOIDS4_fil), ROW_AXIS(SEQUENCE(ROW_I)), 
                SERIES_ID(ID),
                PAYLOAD(FIELDS(Magnitude), CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line'))], IMAGE('png')));

,ID,ROW_I,IMAGE
1,1,0,...


<img id="sine filter" src="images/sine_filter.png" alt="sine filter" width="400" />
<p style = 'font-size:16px;font-family:Arial'>sine wave of filter</p>

In [39]:
/*************************************************************************/
/*      Convolve Filter Kernel with the Original Signal                 */
/* Orignal Signal contained sinusoids at: 2, 25,50, 100HZ and noise     */
/*        Filter should erase everything except 100HZ                  */
/*************************************************************************/

EXECUTE FUNCTION INTO VOLATILE ART(ApplyLowPass) 
TD_CONVOLVE(SERIES_SPEC(TABLE_NAME(SINUSOIDS4_noise), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
                        PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))),
            SERIES_SPEC(TABLE_NAME(SINUSOIDS4_fil), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
                        PAYLOAD( FIELDS(MAGNITUDE), CONTENT(REAL))),
            INPUT_FMT(INPUT_MODE(MATCH)));

Success: 1024 rows affected

In [40]:
SHOW TABLE ApplyLowPass;

,Request Text
1,"CREATE MULTISET VOLATILE TABLE DEMO_USER.ApplyLowPass ,FALLBACK , CHECKSUM = DEFAULT, DEFAULT MERGEBLOCKRATIO, MAP = TD_MAP2, ART, LOG ( ID INTEGER NOT NULL, ROW_I INTEGER NOT NULL, REAL_MAGNITUDE FLOAT, IMAG_MAGNITUDE FLOAT) NO PRIMARY INDEX ON COMMIT PRESERVE ROWS;"


In [41]:
/*************************************************************************/
/* Verify that the filtered Signal has the desired freq characteristics  */
/*                             using the DFFT                            */
/*************************************************************************/
--DROP TABLE DfftFilteredSins4;
EXECUTE FUNCTION INTO VOLATILE ART(DfftFilteredSins4) 
TD_DFFT(SERIES_SPEC(TABLE_NAME(ApplyLowPass), SERIES_ID(ID), ROW_AXIS(SEQUENCE(ROW_I)),
                    PAYLOAD( FIELDS(REAL_MAGNITUDE), CONTENT(REAL)) ),
        FUNC_PARAMS( FREQ_STYLE("K_INTEGRAL"), HUMAN_READABLE(1) ),
  OUTPUT_FMT( CONTENT(AMPL_PHASE_RADIANS) ) );


Success: 1026 rows affected

In [42]:
EXECUTE FUNCTION
TD_Plot
(   SERIES_SPEC(TABLE_NAME(DfftFilteredSins4), ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(ID),
                PAYLOAD(FIELDS(Amplitude_real_Magnitude),CONTENT(REAL)) ),
    FUNC_PARAMS( PLOTS[( TYPE('line') )], IMAGE('png'))
);

,ID,ROW_I,IMAGE
1,1,0,...


<img id="filtered signal" src="images/convolve.png" alt="filtered signal" width="400" />
<p style = 'font-size:16px;font-family:Arial'>Sine wave with all frequencies filtered except 100Hz</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Clean up</b>


In [9]:
Drop TABLE genData;

In [9]:
DROP TABLE GenDataSinusoids;

In [9]:
DROP TABLE GeneratedData;

In [9]:
DROP TABLE SINUSOIDS4;

In [9]:
DROP TABLE SINUSOIDS4_noise;

In [ ]:
Drop TABLE SINUSOIDS4_fil;

<p style = 'font-size:16px;font-family:Arial'>In this demo we have seen how we can transform timeseries to frequency domain to get significant frequencies and vice-versa and how we can filter out the frequencies from our signal series.</p>
<footer style="paddinga:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2022 Teradata. All Rights Reserved</footer>